Loading dataset using Tensorflow Dataset API

In [1]:
import tensorflow_datasets as tfds

In [2]:
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', 
                                        split=(tfds.Split.TRAIN, tfds.Split.TEST),
                                        as_supervised=True,
                                        with_info=True)

Dl Completed...: 100%|██████████| 1/1 [11:22<00:00, 682.52s/ url]


Dataset imdb_reviews downloaded and prepared to C:\Users\User\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


Now check some examples from the dataset.

In [3]:
for review, label in tfds.as_numpy(ds_train.take(5)):
    print(review.decode()[0:50],'\t', label)

This was an absolutely terrible movie. Don't be lu 	 0
I have been known to fall asleep during films, but 	 0
Mann photographs the Alberta Rocky Mountains in a  	 0
This is the kind of film for a snowy Sunday aftern 	 1
As others have mentioned, all the women that go nu 	 1


Now we need to apply BERT tokenizer to use pre-trained tokenizer.

In [1]:
from transformers import BertTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Now we need to prepare the data for BERT model.

Input IDs – The input ids are often the only required parameters to be passed to the model as input. Token indices, numerical representations of tokens building the sequences that will be used as input by the model.

Attention mask – Attention Mask is used to avoid performing attention on padding token indices. Mask value can be either 0 or 1, 1 for tokens that are NOT MASKED, 0 for MASKED tokens.

Token type ids – It is used in use cases like sequence classification or question answering. As these require two different sequences to be encoded in the same input IDs. Special tokens, such as the classifier[CLS] and separator[SEP] tokens are used to separate the sequences.

<center><img src="img1.png"/></center>
<center>Source: BERT Input/Output(https://pysnacks.com/machine-learning/bert-text-classification-with-fine-tuning/)</center>

The encode_plus function of the tokenizer class will tokenize the raw input.

In [ ]:
max_length = 512
def convert_to_feature(review):
    return tokenizer.encode_plus(review,
                    add_special_tokens=True, # add [CLS], [SEP]
                    max_length=max_length, # max length of the text that can go to BERT
                    pad_to_max_length=True, # add [PAD] tokens 
                    return_attention_mask=True) # add attention mask to not focus on pad tokens

Following helper functions are transforming raw data to an appropriate format to feed into the BERT model.